In [3]:
# p141 실습
# 아래는 통신회사의 고객 이탈 데이터이다. 고객의 이탈 여부를 예측하기 위한 그래디언트 부스팅 분석을 실시하고 결과를 해석하시오.
# 데이터 : 통신고객이탈.csv (변수 : 15개, 자료 수 : 930게)

# 1. 데이터 구성하기
# 2. 모델 생성 및 결론 도출 - 그래디언트 부스팅
#-Parameter 조정 및 모델 평가
#-결론도출

# 3. 모델 생성 및 결론 도출-XGBoost
#-Parameter 입력 및 모델 평가
#-결론 도출

import pandas as pd # 데이터 구성 : Series, DataFrame
import matplotlib # 데이터 시각화
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from statsmodels.api import Logit

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False
# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'

In [5]:
# 데이터 구성하기
df_raw = pd.read_csv("/home/piai/다운로드/2주차 실습파일/2. Big Data 분석/통신고객이탈.csv")
df_raw.head()

,CHURN,CUSTOMER_ID,GENDER,AGE,CHARGE_TYPE,HANDSET,USAGE_BAND,SERVICE_DURATION,DROPPED_CALLS,PEAK_CALLS_NO,PEAK_CALLS_TIME,WEEKEND_CALLS_NO,WEEKEND_CALLS_TIME,TOTAL_CALLS_NO,TOTAL_CALLS_TIME
0,Active,K102990,F,31,CAT 100,ASAD170,Med,33.33,6,218,379.8,9,21.6,366,632.4
1,Active,K103280,M,27,CAT 100,S50,Med,30.87,10,373,656.4,11,28.0,411,810.4
2,Active,K103660,M,27,CAT 100,ASAD170,Med,49.40,11,260,582.0,0,0.0,265,614.1
3,Active,K103730,M,61,CAT 100,ASAD170,Med,48.60,1,294,661.8,14,101.5,326,844.0
4,Active,K104560,F,52,CAT 100,S50,Med,56.03,0,221,555.6,7,25.0,274,673.0


In [6]:
# 데이터 구성하기 - 확인
df_raw.info()

# 실수 4개, 정수 5개, 6개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CHURN               930 non-null    object 
 1   CUSTOMER_ID         930 non-null    object 
 2   GENDER              930 non-null    object 
 3   AGE                 930 non-null    int64  
 4   CHARGE_TYPE         930 non-null    object 
 5   HANDSET             930 non-null    object 
 6   USAGE_BAND          930 non-null    object 
 7   SERVICE_DURATION    930 non-null    float64
 8   DROPPED_CALLS       930 non-null    int64  
 9   PEAK_CALLS_NO       930 non-null    int64  
 10  PEAK_CALLS_TIME     930 non-null    float64
 11  WEEKEND_CALLS_NO    930 non-null    int64  
 12  WEEKEND_CALLS_TIME  930 non-null    float64
 13  TOTAL_CALLS_NO      930 non-null    int64  
 14  TOTAL_CALLS_TIME    930 non-null    float64
dtypes: float64(4), int64(5), object(6)
memory usage: 109.1+ K

In [7]:
print("Data 구조:", df_raw.shape) # 데이터 수 체크
print("변수:", df_raw.columns) # 열 체크

Data 구조: (930, 15)
변수: Index(['CHURN', 'CUSTOMER_ID', 'GENDER', 'AGE', 'CHARGE_TYPE', 'HANDSET',
       'USAGE_BAND', 'SERVICE_DURATION', 'DROPPED_CALLS', 'PEAK_CALLS_NO',
       'PEAK_CALLS_TIME', 'WEEKEND_CALLS_NO', 'WEEKEND_CALLS_TIME',
       'TOTAL_CALLS_NO', 'TOTAL_CALLS_TIME'],
      dtype='object')


In [12]:
# 데이터 전처리 - 결측치 확인 및 처리

# 결측치 확인
df_raw.isnull().sum(axis = 0)

CHURN                 0
CUSTOMER_ID           0
GENDER                0
AGE                   0
CHARGE_TYPE           0
HANDSET               0
USAGE_BAND            0
SERVICE_DURATION      0
DROPPED_CALLS         0
PEAK_CALLS_NO         0
PEAK_CALLS_TIME       0
WEEKEND_CALLS_NO      0
WEEKEND_CALLS_TIME    0
TOTAL_CALLS_NO        0
TOTAL_CALLS_TIME      0
dtype: int64

In [10]:
print("Unique values in 'CHURN':", df_raw['CHURN'].unique())
# 유니크 값 확인으로 CHURN 열의 값을 다시 확인한다

df_raw['CHURN'] = df_raw['CHURN'].str.strip() # 앞 뒤 공백 제거를 한다.
df_raw['CHURN'] = df_raw['CHURN'].map({'Active': 0, 'Churned': 1}) # 매핑으로 active를 0으로 churned를 1로 바꾼다.

print(df_raw)

Unique values in 'CHURN': ['Active' 'Churned']
     CHURN CUSTOMER_ID GENDER  AGE CHARGE_TYPE  HANDSET USAGE_BAND  \
0        0     K102990      F   31     CAT 100  ASAD170        Med   
1        0     K103280      M   27     CAT 100      S50        Med   
2        0     K103660      M   27     CAT 100  ASAD170        Med   
3        0     K103730      M   61     CAT 100  ASAD170        Med   
4        0     K104560      F   52     CAT 100      S50        Med   
..     ...         ...    ...  ...         ...      ...        ...   
925      1     K413330      M   19    Play 100    SOP10        Med   
926      1     K413410      F   59    Play 100    SOP20        Med   
927      1     K415490      F   33    Play 300      S50        Med   
928      1     K415880      F   25    Play 300      S50        Med   
929      1     K416650      F   30    Play 300    SOP10        Med   

     SERVICE_DURATION  DROPPED_CALLS  PEAK_CALLS_NO  PEAK_CALLS_TIME  \
0               33.33              6    

In [11]:
# 데이터 전처리 - 데이터 분할(train, tes)t 데이터 7 : 3 비율로 분할)

# train_test_split(데이터, test_size = test 데이터 비율, randim_state : 랜덤)
df_train, df_test = train_test_split(df_raw, test_size = 0.3, random_state = 1234)

print("train data size: {}".format(df_train.shape))
print("test data size: {}".format(df_test.shape))

# 본 과정에서는 동일한 분석 결과 생성을 위해 random_state = 1234로 지정
# 실제 과제에서는 random_state에 특정값을 지정하지 않는 것이 바람직함.

# train 데이터 2623건, test 데이터 1125건

train data size: (651, 15)
test data size: (279, 15)


In [ ]:
# 그래디언트 부스팅 모델 생성: GradientBoostingClassifier
gb_uncust = GradientBoostingClassifier(random_state=1234)
gb_uncust.fit(df_train_x, df_train_y)

# train 모델 정확도
print("Accuracy on training set: {:.3f}".format(gb_uncust.score (df_train_x, df_train_y)))
# test 모델 정확도
print("Accuracy on test set: {:.3f}".format(gb_uncust.score (df_test_x, df_test_y)))